# Mapping Notebook

## Import modules

In [1]:
import os
import pandas as pd

## Creating input file for download and processing script

Downloaded from [**here**](https://www.ebi.ac.uk/biostudies/arrayexpress/studies/E-MTAB-12916?query=E-MTAB-12916)

In [3]:
data_path = '../../raw_data/'
meta_data_file = 'E-MTAB-12916.sdrf.txt'

In [4]:
meta_data = pd.read_csv(os.path.join(data_path, meta_data_file), sep='\t')
meta_data

,Source Name,Comment[ENA_SAMPLE],Comment[BioSD_SAMPLE],Characteristics[organism],Characteristics[age],Characteristics[developmental stage],Characteristics[sex],Characteristics[individual],Characteristics[organism part],Characteristics[disease],...,Assay Name,Technology Type,Comment[ENA_EXPERIMENT],Scan Name,Comment[SUBMITTED_FILE_NAME],Comment[ENA_RUN],Comment[FASTQ_URI],Comment[read_index],Comment[read_type],Factor Value[organism part]
0,HCAHeart9508627,ERS15408104,SAMEA113412973,Homo sapiens,55 to 60,adult,Male,D3,heart left ventricle,normal,...,HCAHeart9508627,sequencing assay,ERX10811380,HCAHeart9508627_S1_L001_I1_001.fastq.gz,HCAHeart9508627_S1_L001_I1_001.fastq.gz,ERR11403589,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,index1,sample_barcode,heart left ventricle
1,HCAHeart9508627,ERS15408104,SAMEA113412973,Homo sapiens,55 to 60,adult,Male,D3,heart left ventricle,normal,...,HCAHeart9508627,sequencing assay,ERX10811380,HCAHeart9508627_S1_L001_I2_001.fastq.gz,HCAHeart9508627_S1_L001_I2_001.fastq.gz,ERR11403589,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,index2,sample_barcode,heart left ventricle
2,HCAHeart9508627,ERS15408104,SAMEA113412973,Homo sapiens,55 to 60,adult,Male,D3,heart left ventricle,normal,...,HCAHeart9508627,sequencing assay,ERX10811380,HCAHeart9508627_S1_L001_R1_001.fastq.gz,HCAHeart9508627_S1_L001_R1_001.fastq.gz,ERR11403589,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,read1,"cell_barcode,umi_barcode",heart left ventricle
3,HCAHeart9508627,ERS15408104,SAMEA113412973,Homo sapiens,55 to 60,adult,Male,D3,heart left ventricle,normal,...,HCAHeart9508627,sequencing assay,ERX10811380,HCAHeart9508627_S1_L001_R2_001.fastq.gz,HCAHeart9508627_S1_L001_R2_001.fastq.gz,ERR11403589,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,read2,single,heart left ventricle
4,HCAHeart9508628,ERS15408105,SAMEA113412974,Homo sapiens,60 to 65,adult,Male,D7,right cardiac atrium,normal,...,HCAHeart9508628,sequencing assay,ERX10811381,HCAHeart9508628_S1_L001_I1_001.fastq.gz,HCAHeart9508628_S1_L001_I1_001.fastq.gz,ERR11403590,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,index1,sample_barcode,right cardiac atrium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,HCAHeartST13189996,ERS15408149,SAMEA113413018,Homo sapiens,70 to 75,adult,Female,AV13,atrioventriculeft cardiac atriumr node,normal,...,HCAHeartST13189996,sequencing assay,ERX10811425,HCAHeartST13189996_S1_L001_R2_001.fastq.gz,HCAHeartST13189996_S1_L001_R2_001.fastq.gz,ERR11403634,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,read2,single,atrioventriculeft cardiac atriumr node
184,HCAHeartST13189997,ERS15408150,SAMEA113413019,Homo sapiens,70 to 75,adult,Female,AV13,atrioventriculeft cardiac atriumr node,normal,...,HCAHeartST13189997,sequencing assay,ERX10811426,HCAHeartST13189997_S1_L001_I1_001.fastq.gz,HCAHeartST13189997_S1_L001_I1_001.fastq.gz,ERR11403635,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,index1,sample_barcode,atrioventriculeft cardiac atriumr node
185,HCAHeartST13189997,ERS15408150,SAMEA113413019,Homo sapiens,70 to 75,adult,Female,AV13,atrioventriculeft cardiac atriumr node,normal,...,HCAHeartST13189997,sequencing assay,ERX10811426,HCAHeartST13189997_S1_L001_I2_001.fastq.gz,HCAHeartST13189997_S1_L001_I2_001.fastq.gz,ERR11403635,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,index2,sample_barcode,atrioventriculeft cardiac atriumr node
186,HCAHeartST13189997,ERS15408150,SAMEA113413019,Homo sapiens,70 to 75,adult,Female,AV13,atrioventriculeft cardiac atriumr node,normal,...,HCAHeartST13189997,sequencing assay,ERX10811426,HCAHeartST13189997_S1_L001_R1_001.fastq.gz,HCAHeartST13189997_S1_L001_R1_001.fastq.gz,ERR11403635,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,read1,"cell_barcode,umi_barcode",atrioventriculeft cardiac atriumr node


In [28]:
meta_data = meta_data[['Source Name', 'Comment[FASTQ_URI]', 'Comment[SUBMITTED_FILE_NAME]']]
meta_data.to_csv('../../raw_data/downloads_table.csv', header=None)
meta_data

,Source Name,Comment[FASTQ_URI],Comment[SUBMITTED_FILE_NAME]
0,HCAHeart9508627,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeart9508627_S1_L001_I1_001.fastq.gz
1,HCAHeart9508627,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeart9508627_S1_L001_I2_001.fastq.gz
2,HCAHeart9508627,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeart9508627_S1_L001_R1_001.fastq.gz
3,HCAHeart9508627,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeart9508627_S1_L001_R2_001.fastq.gz
4,HCAHeart9508628,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeart9508628_S1_L001_I1_001.fastq.gz
...,...,...,...
183,HCAHeartST13189996,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeartST13189996_S1_L001_R2_001.fastq.gz
184,HCAHeartST13189997,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeartST13189997_S1_L001_I1_001.fastq.gz
185,HCAHeartST13189997,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeartST13189997_S1_L001_I2_001.fastq.gz
186,HCAHeartST13189997,ftp://ftp.sra.ebi.ac.uk/vol1/run/ERR114/ERR114...,HCAHeartST13189997_S1_L001_R1_001.fastq.gz


## Index file creation

**Comprehensive gene annotation CHR:**
- It contains the comprehensive gene annotation on the reference chromosomes only

**Long non-coding RNA gene annotation CHR:**
- It contains the comprehensive gene annotation of lncRNA genes on the reference chromosomes

**Genome sequence (GRCh38.p14) ALL:**
- Nucleotide sequence of the GRCh38.p14 genome assembly version on all regions, including reference chromosomes, scaffolds, assembly patches and haplotypes
- The sequence region names are the same as in the GTF/GFF3 files

downloaded from [**here**](https://www.gencodegenes.org/human/)

In [4]:
%%bash
index_input="../../reference_data/"
index_output="../../kb_index/"

if [ ! -d "$index_output" ]; then
    mkdir -p "$index_output"
fi

kb ref -i "$index_output/transcriptome.idx" \
        -g "$index_output/transcripts_to_genes.txt" \
        -f1 "$index_output/cdna.fa" \
        "index_input/GRCh38.p14.genome.fa.gz" \
        "index_input/gencode.v45.chr_patch_hapl_scaff.annotation.gtf.gz" \
        "index_input/gencode.v45.long_noncoding_RNAs.gtf.gz" \
        

[2024-04-11 12:05:39,853]    INFO [ref] Skipping cDNA FASTA generation because ../../kb_index//cdna.fa already exists. Use --overwrite flag to overwrite
[2024-04-11 12:05:39,853]    INFO [ref] Skipping kallisto index because ../../kb_index//transcriptome.idx already exists. Use the --overwrite flag to overwrite.


## Mapping

In [ ]:
%%bash

metadata='../../raw_data/downloads_table.csv'
output_path='../../mapping/'
index_output="../../kb_index/"

# Check if the file exists
if [ ! -f "$metadata" ]; then
    echo "File not found: $metadata"
    exit 1
fi

# Read the CSV file and download files for each sample
while IFS=, read -r index sample url file_name; do
    # Create the directory for the sample
    mkdir -p "$output_path/$sample"
    
    # Download the file using axel
    axel -n 10 --output="$output_path/$sample/$file_name" "$url"
    
    # Keep track of the downloaded files
    downloaded_files+=("$output_path/$sample/$file_name")
    
    # Check if 4 files have been downloaded for the current sample
    if [ ${#downloaded_files[@]} -eq 4 ]; then
        echo "Processing files for sample: $sample"
        
        # Run kb count command to process the downloaded files
        kb count -i "$index_output/transcriptome.idx" -g "$index_output/transcripts_to_genes.txt" -x 10XV3 \
            --h5ad \
            "$output_path/$sample"/*.fastq.gz \
            -o "$output_path/$sample/"
        
        echo "Deleting downloaded files for sample: $sample"
        # Delete only the downloaded files
        rm "${downloaded_files[@]}"
        # Clear the downloaded files array for the next sample
        downloaded_files=()
    fi
done < "$metadata"
